# 1. Import Dependencies

In [1]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
from imblearn.over_sampling import SMOTE
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback

# 2. Data Preparation

In [2]:
df = pd.read_csv('push_up_bottom_position_fix.csv')

In [3]:
jumlah_kelas_0 = df[df['class'] == 0].shape[0]
jumlah_kelas_1 = df[df['class'] == 1].shape[0]

print("Jumlah data kelas 0:", jumlah_kelas_0)
print("Jumlah data kelas 1:", jumlah_kelas_1)

Jumlah data kelas 0: 1611
Jumlah data kelas 1: 4653


In [4]:
X = df.drop('class', axis=1)
y = df['class']

In [5]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 3. Modelling

In [60]:
model = tf.keras.Sequential([
    layers.Dense(64, input_dim=132, activation='relu', kernel_regularizer=regularizers.l2(0.05)),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.05)),
    layers.Dense(1, activation='sigmoid')  
])


In [61]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [62]:
class AccuracyStopCallback(Callback):
    def __init__(self, target_accuracy=0.99):
        super(AccuracyStopCallback, self).__init__()
        self.target_accuracy = target_accuracy
        self.val_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= self.target_accuracy and logs.get('val_accuracy') >= self.target_accuracy:
            print(f"\nReached target accuracy of {self.target_accuracy}, stopping training!")
            self.model.stop_training = True

In [63]:
callback = AccuracyStopCallback(target_accuracy=0.97)

In [64]:
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2, callbacks=[callback])

Epoch 1/20
233/233 [==============================] - 3s 11ms/step - loss: 2.3148 - accuracy: 0.9475 - val_loss: 0.4822 - val_accuracy: 0.9860
Epoch 2/20
229/233 [============================>.] - ETA: 0s - loss: 0.2776 - accuracy: 0.9967
Reached target accuracy of 0.97, stopping training!
233/233 [==============================] - 2s 10ms/step - loss: 0.2763 - accuracy: 0.9968 - val_loss: 0.2019 - val_accuracy: 1.0000


In [65]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_head_position.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmp1luek_71\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmp1luek_71\assets


# 4. Testing

In [66]:
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

landmarks = ["class"]
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

vid_path = "test.mp4"
cap = cv2.VideoCapture(vid_path)
current_stage = ''

# Initiate Pose Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # Streaming the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to the desired window size
        frame = cv2.resize(frame, (720, 600))

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detections
        results = pose.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        try:
            if results.pose_landmarks:
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                row = np.expand_dims(row, axis=0)  # Add a batch dimension
                prediction = model.predict(row)
                print(f'Result: {prediction[0][0]}')
                
                if prediction[0][0] > 0.5:
                    current_stage = 'Correct'
                elif prediction[0][0] <= 0.5:
                    current_stage = 'Wrong'

                cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

                cv2.putText(image, 'CLASS', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(prediction[0][0], 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except Exception as e:
            print(f"Error: {e}")


        # Stream video result
        cv2.imshow("Raw Cam Feed", image)

        # Press 'q' to stop the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


Result: 0.46493127942085266
Result: 0.46218857169151306
Result: 0.4572839140892029
Result: 0.4524644613265991
Result: 0.4481213688850403
Result: 0.4482671916484833
Result: 0.4482751488685608
Result: 0.4500003755092621
Result: 0.44950661063194275
Result: 0.4610000550746918
Result: 0.4653167724609375
Result: 0.4633539915084839
Result: 0.4648382067680359
Result: 0.46872299909591675
Result: 0.4676916003227234
Result: 0.4663572311401367
Result: 0.45688819885253906
Result: 0.44989249110221863
Result: 0.4355205297470093
Result: 0.421915739774704
Result: 0.4254249036312103
Result: 0.41508856415748596
Result: 0.4075659215450287
Result: 0.3966848850250244
Result: 0.39084482192993164
Result: 0.3948352336883545
Result: 0.3916974365711212
Result: 0.3995460271835327
Result: 0.3963199257850647
Result: 0.39336052536964417
Result: 0.3961240351200104
Result: 0.39718854427337646
Result: 0.4002985954284668
Result: 0.39235273003578186
Result: 0.39569029211997986
Result: 0.3942117393016815
Result: 0.3907362